In [1]:
import pandas as pd

In [2]:
def clean_dataframe(df, registry, start, end, verbose=False):

    registry['DateStop'] = pd.to_datetime(registry['DateStop'])
    registry = registry[registry['Pollutant'] == pollutant]
    active = registry[(pd.isna(registry.DateStop)) | (registry.DateStop > pd.to_datetime(end))]

    df['Date'] = pd.to_datetime(df['Date'])

    data = df[(df.Date > pd.to_datetime(start)) & (df.Date < pd.to_datetime(end))]
    data = data.groupby('IDStation', as_index=False).apply(lambda g: g.mean(skipna=True))
    data['IDStation'] = data['IDStation'].astype(int)


    active = registry[(pd.isna(registry.DateStop)) | (registry.DateStop > pd.to_datetime(end))]

    clean_df = pd.merge(data, active[['IDStation', 'Latitude', 'Longitude', 'Altitude']], on='IDStation', how='inner')

    if verbose:
        print(clean_df.shape)
        print(clean_df.info())        

    return clean_df

In [3]:
pollutants = ['NO2', 'NOx', 'PM2.5', 'PM10']

In [4]:
registry = pd.read_csv('./data/registry.csv')

In [5]:
for pollutant in pollutants:

    df = pd.read_csv('./data/raw/{}_sit_monthly.csv'.format(pollutant)).drop('Unnamed: 0', axis=1)

    clean_df_2019 = clean_dataframe(df, registry, 'March 2019', 'July 2019', verbose=False)

    clean_df_2019.to_csv('./data/2019/{}_2019.csv'.format(pollutant))

    clean_df_2020 = clean_dataframe(df, registry, 'March 2020', 'July 2020', verbose=False)

    clean_df_2020.to_csv('./data/2020/{}_2020.csv'.format(pollutant))

    diff_df = pd.merge(clean_df_2019, clean_df_2020, on=['IDStation', 'Latitude', 'Longitude', 'Altitude'], suffixes=['_2019', '_2020'])

    diff_df['{}_Delta'.format(pollutant)] = diff_df[pollutant+ '_2019'] - diff_df[pollutant + '_2020']

    diff_df = diff_df[['IDStation', pollutant+ '_2019', pollutant+ '_2020', pollutant+ '_Delta', 'Latitude', 'Longitude']]

    diff_df = diff_df.dropna()

    diff_df.to_csv('./data/{}.csv'.format(pollutant))